In [ ]:
from flask import Flask
from flask import request
from flask import Response
from flask import stream_with_context
from streamer import Streamer
import socket
import pickle
import struct
import cv2
from flask import render_template

app = Flask( __name__ )
streamer = Streamer()
ip = '172.30.1.52'
port = 50001
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client_socket.connect((ip, port))
streamer.run(0)
@app.route('/stream')
def stream():
    #p_address = flask.request.remote_addr
    src = request.args.get( 'src', default = 0, type = int )
    try :
        return Response(stream_with_context( stream_gen( src ) ),mimetype='multipart/x-mixed-replace; boundary=frame' )
    except Exception as e :
        print('[wandlab] ', 'stream error : ',str(e))
        
        
def stream_gen( src ):   
    try :  
        
        while True :
            frame = streamer.bytescode()
            socket_frame = streamer.socket_frame()
            #print("socket_frame",socket_frame)
            client_socket.sendall(struct.pack(">L", len(socket_frame)) + socket_frame)
            #print("socket_frame send 성공")
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
    except GeneratorExit :
        print( '[wandlab]', 'disconnected stream' )
        #treamer.stop()
if __name__ == '__main__' :
    app.run(host='172.30.1.6', port=5000 )

[wandlab]  OpenCL :  False
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://172.30.1.6:5000
Press CTRL+C to quit
172.30.1.6 - - [13/Feb/2023 09:19:04] "GET / HTTP/1.1" 404 -
172.30.1.6 - - [13/Feb/2023 09:19:04] "GET /favicon.ico HTTP/1.1" 404 -
172.30.1.6 - - [13/Feb/2023 09:19:06] "GET /stream HTTP/1.1" 200 -
